In [32]:
import os

In [33]:
from langchain_community.document_loaders import PyMuPDFLoader

In [34]:
loader = PyMuPDFLoader("industry_laws.pdf")
pages = loader.load()

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
# 문서를 문장으로 분리
## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)

In [37]:
from langchain.embeddings import HuggingFaceEmbeddings

In [38]:
# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

In [39]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

Vectorstore created and persisted


In [40]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
model = ChatOllama(model="llama3.1", temperature=0)

In [41]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

In [45]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성
template = '''상대방의 요청에 최대한 자세하고 객관적으로 답하자. 
모든 대답은 한국어(Korean)으로 대답하자.
사고 경위가 입력되면 그 사고 경위를 이해하고,
그와 관련된 법령 5가지를 찾아 관련도가 높은 순서대로 나열해서 보고서 형식으로 말하자:



{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [46]:
# Chain 실행
query = "현장소장 외 5명, 크레인기사1명이 공사 시작 첫날 2019.10.28일 11시 50분경 O구에 위치한 해양경찰서 OO파출소 앞에서 붐의 끝부분을  수신호에 맞지 않게 들다가 크레인이 기울어지며 크레인과 붐을 연결하고 있던 슬링밸트가 풀어지며 피해자가  파일과 붐사이에 끼여서 OO대학병원 응급실에 가서 응급수술을 하다가 사망함."
answer = rag_chain.invoke(query)
print("Query:", query)
print("Answer:", answer)

Query: 현장소장 외 5명, 크레인기사1명이 공사 시작 첫날 2019.10.28일 11시 50분경 O구에 위치한 해양경찰서 OO파출소 앞에서 붐의 끝부분을  수신호에 맞지 않게 들다가 크레인이 기울어지며 크레인과 붐을 연결하고 있던 슬링밸트가 풀어지며 피해자가  파일과 붐사이에 끼여서 OO대학병원 응급실에 가서 응급수술을 하다가 사망함.
Answer: **보고서**

**사고 경위**

2019년 10월 28일, O구 해양경찰서 OO파출소 앞에서 크레인 작업이 시작되었습니다. 현장소장 외 5명과 크레인 기사가 함께 작업을 수행했습니다. 그러나 첫날인 11시 50분 경에, 크레인이 기울어지며 슬링밸트가 풀어졌습니다. 이로 인해 피해자가 파일과 붐사이에 끼여서 OO대학병원 응급실로 옮겨져 응급수술을 받았지만 사망했습니다.

**관련 법령 5가지**

1. **제3관 이동식 크레인 제147조(설계기준 준수)**: 사업주는 이동식 크레인을 사용하는 경우에 그 이동식 크레인이 넘어지거나 그 이동식 크레인의 구조 부분을 구성하는 강재 등이 변형되거나 부러지는 일 등을 방지하기 위하여 해당 이동식 크레인의 설계기준(제조자가 제공하는 사용설명서)을 준수하여야 한다.
2. **제3관 이동식 크레인 제148조(안전밸브의 조정)**: 사업주는 유압을 동력으로 사용하는 이동식 크레인의 과도한 압력상승을 방지하기 위한 안전밸브를 설치하고, 그 밸브의 조정을 철저히 하여야 한다.
3. **제147조(설계기준 준수) 제정개정사**: 2024년 6월 28일부터 적용되는 법령으로, 사업주는 이동식 크레인의 설계기준을 준수하여야 하며, 그 이동식 크레인이 넘어지거나 구조 부분이 변형되거나 부러지는 일 등을 방지하기 위하여 해당 이동식 크레인의 설계기준을 준수해야 한다.
4. **제148조(안전밸브의 조정) 제정개정사**: 2024년 6월 28일부터 적용되는 법령으로, 사업주는 유압을 동력으로 사용하는 이동식 크레인의 과도한 압력상승을 방지하기 위한 안전밸브를 설치하고, 그 밸브의 조정을 